## Bedrock model integration with Langchain Agents

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

In this notebook, we will demonstrate the use of `plan-and-execute` agent that determines the complete order of actions initially, then implement them all without updating the plan.


In [1]:
import json
import os
import sys
import boto3

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ['SERPAPI_API_KEY'] = "<YOUR_SERP_API_KEY>" # Required for the search tool

In [2]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import AgentType
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock
from langchain import LLMMathChain
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

Load two unique LLM objects with unique model parameters, one for the agent, and the other for the math chain.

A unique llm with stop sequences for the math chain helps prevent Claude from being overly verbose while running the math chain.

Additionally, we'll adapt the default template to better fit Claude, as the default Langchain templates are not well fit by default, then append the newly constructed tool to our list of tools.

In [4]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

react_agent_llm = ChatBedrock(model_id=modelId, client=bedrock_client)
math_chain_llm = ChatBedrock(model_id=modelId, client=bedrock_client)
tools = load_tools(["wikipedia"], llm=react_agent_llm)

llm_math_chain =  LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
{{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [5]:
prompt_template = """Answer the following questions as best you can. 
You have access to the following tools:\n\n{tools}\n\n
Use the following format:\n\nQuestion: the input question you must answer\n
Thought: you should always think about what to do\n
Action: the action to take, should be one of [{tool_names}]\n
Action Input: the input to the action\nObservation: the result of the action\n... 
(this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n\nBegin!\n\n
Question: {input}\nThought:{agent_scratchpad}
"""

In [6]:
from langchain_core.prompts import PromptTemplate
react_agent = create_react_agent(react_agent_llm, 
                               tools,
                                 PromptTemplate.from_template(prompt_template)
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )

In [7]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"

In [8]:
agent_executor = AgentExecutor(
agent=react_agent, 
tools=tools,
verbose=True,
handle_parsing_errors=True,
max_iterations = 10 # useful when agent is stuck in a loop
)

In [9]:
agent_executor.invoke({"input": question}) 




> Entering new AgentExecutor chain...
Question: What is Amazon SageMaker? What is the launch year multiplied by 2

Thought: To answer this question, I need to first find information about what Amazon SageMaker is from a reliable source like Wikipedia. After that, I need to determine the launch year and multiply it by 2.

Action: wikipedia
Action Input: amazon sagemaker
Page: Amazon SageMaker
Summary: Amazon SageMaker is a cloud based machine-learning platform that allows the creation, training, and deployment by developers of machine-learning (ML) models on the cloud. It can be used to deploy ML models on embedded systems and edge-devices. SageMaker was launched in November 2017.

Page: Amazon (company)
Summary: Amazon.com, Inc., doing business as Amazon (, AM-ə-zon; UK also , AM-ə-zən), is an American multinational technology company, engaged in e-commerce, cloud computing, online advertising, digital streaming, and artificial intelligence. It is considered one of the Big Five Ameri

{'input': 'What is Amazon SageMaker? What is the launch year multiplied by 2',
 'output': 'Amazon SageMaker is a cloud-based machine learning platform launched by Amazon in November 2017. The launch year, 2017, multiplied by 2 is 4034.'}